01_descriptive_analysis.ipynb

## 1. Churn & Revenue Baselines

* Quarterly & monthly churn rate overall and by segment (SME, mid‑market, enterprise)

* Average and total ARR per segment (using subscription plan pricing)

## 2. Customer Tenure & Lifecycle

* Distribution of tenure at churn date

* Time from signup → first subscription

* Distribution of subscription lengths (renewal cadence)

## 3. Usage & Support Summaries

* Average logins/events per user-month by retained vs. churned

* Mean & median # support tickets per customer by segment

## 4. Marketing Touchpoints

* Volume of acquisitions by channel (email, webinar, demo)

* Basic conversion funnel (events → first subscription)